# 플랫폼 업로드를 쉽게하기 위한 로컬 개발 코드
- T3Q.ai(T3Q.cep + T3Q.dl): 빅데이터/인공지능 통합 플랫폼
- 플랫폼 업로드를 쉽게하기 위하여 로컬에서 2개의 파일로 나누어 코드를 개발한다.
- 파일 1_1: 전처리와 학습모델을 개발하는 코드, 전처리 객체나 성능이 검증된 학습모델 객체를 저장한다.
- 파일 1_2: 학습모델을 통해 추론(예측)을 개발하는 코드, 전처리 객체나 학습모델 객체를 불러와서 추론을 수행한다. 

- 파일 1_1(파일명): 1_1_local_platform_iris_classification_preprocess_train_pytorch.ipynb
- 파일 1_2(파일명): 1_2_local_platform_iris_classification_inference_pytorch.ipynb

### 전처리 객체 또는 학습모델 객체
- 전처리 객체나 학습모델 객쳬는 meta_data 폴더 아래에 저장한다.

### 데이터 세트(학습 데이터/테스트 데이터)
- 학습과 테스트에 사용되는 데이터를 나누어 관리한다.
- 학습 데이터: dataset 폴더 아래에 저장하거나 dataset.zip 파일 형태로 저장한다.
- 테스트 데이터: test_dataset 폴더 아래에 저장하거나 test_dataset.zip 파일 형태로 저장한다.

### 로컬 개발 워크플로우(workflow)  
- 로컬 개발 워크플로우를 다음의 4단계로 분리한다.

1. **데이터 세트 준비(Data Setup)**
- 로컬 저장소에서 전처리 및 학습에 필요한 학습 데이터 세트를 준비한다.

2. **데이터 전처리(Data Preprocessing)**
- 데이터 세트의 분석 및 정규화(Normalization)등의 전처리를 수행한다.
- 데이터를 모델 학습에 사용할 수 있도록 가공한다.
- 추론과정에서 필요한 경우, 데이터 전처리에 사용된 객체를 meta_data 폴더 아래에 저장한다.

3. **학습 모델 훈련(Train Model)**
- 데이터를 훈련에 사용할 수 있도록 가공한 뒤에 학습 모델을 구성한다. 
- 학습 모델을 준비된 데이터 세트로 훈련시킨다.
- 정확도(Accuracy)나 손실(Loss)등 학습 모델의 성능을 검증한다.
- 학습 모델의 성능 검증 후, 학습 모델을 배포한다.
- 배포할 학습 모델을 meta_data 폴더 아래에 저장한다.

4. **추론(Inference)**
- 저장된 전처리 객체나 학습 모델 객체를 준비한다.
- 추론에 필요한 테스트 데이터 세트를 준비한다.
- 배포된 학습 모델을 통해 테스트 데이터에 대한 추론을 진행한다. 

In [1]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

C:\Users\USER\anaconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **1. 데이터 세트 준비(Data Setup)**

In [2]:
# IRIS dataset를 불러온다.
dataset = pd.read_csv('dataset/iris.csv')

# Species을 numerics로 변환한다.
dataset.loc[dataset.iloc[:,-1]=='Iris-setosa', dataset.columns[-1]] = 0
dataset.loc[dataset.iloc[:,-1]=='Iris-versicolor', dataset.columns[-1]] = 1
dataset.loc[dataset.iloc[:,-1]=='Iris-virginica', dataset.columns[-1]] = 2

## **2. 데이터 전처리(Data Preprocessing)**

In [3]:
train_X, test_X, train_y, test_y = train_test_split(dataset[dataset.columns[0:4]].values, dataset.iloc[:,-1].values, test_size=0.2)

In [4]:
# Pytorch에서 수학적 연산을 가속화하기 위해 tensor data를 사용한다.

train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())

train_y = np.array(train_y, dtype=np.int64)
test_y = np.array(test_y, dtype=np.int64)
train_y = torch.from_numpy(train_y)
test_y = torch.from_numpy(test_y)

## 3. 신경망 구축하기

In [5]:
class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)

        return X    

In [6]:
# Net 클래스를 인스턴스화하고 model 객체를 생성합니다.
# torch.nn.Module의 하위 클래스는 생성한 레이어와 해당 모양 및 매개변수를 보고합니다.

model = Net()
print(model)

Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)


### 4. Optimizer 설정

In [7]:
# loss function: 모델의 예측이 이상적인 출력에서 얼마나 멀리 떨어져 있는지 측정
# optimizer: 학습을 주도(확률적 경사 하강법)
#            학습률, 운동량 외에도 모든 학습 가중치 모음인 net.parameters()도 전달

criterion = nn.CrossEntropyLoss()# cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### 5. 학습 모델 훈련(Train Model)

In [8]:
# loss function과 optimizer를 훈련 루프에 통합

epochs = 100
for epoch in range(1, epochs+1):
    optimizer.zero_grad()
    output = model(train_X)
    loss = criterion(output, train_y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print('number of epoch', epoch, 'loss', loss.data)

number of epoch 10 loss tensor(0.8184)
number of epoch 20 loss tensor(0.6311)
number of epoch 30 loss tensor(0.5842)
number of epoch 40 loss tensor(0.5770)
number of epoch 50 loss tensor(0.5747)
number of epoch 60 loss tensor(0.5743)
number of epoch 70 loss tensor(0.5732)
number of epoch 80 loss tensor(0.5724)
number of epoch 90 loss tensor(0.5720)
number of epoch 100 loss tensor(0.5716)


### 학습 모델 저장

In [9]:
# 경로 지정
# Pytorch에서 모델을 저장할 때, 확장자로 .pt나 .pth를 사용한다.
PATH = "iris_model.pth"

# 저장하기
torch.save(model.state_dict(), PATH)